# 🧩 Add-on Module 6: Machine Learning & Data Preprocessing with Pandas

Machine learning projects start long before model training — data wrangling, cleaning, and feature engineering often take **80%** of your time.

This module focuses on using **Pandas** for end-to-end ML data preparation:
- Handling missing values and outliers
- Encoding categorical variables
- Normalizing and scaling features
- Creating time-based and interaction features
- Integrating Pandas DataFrames with **scikit-learn** pipelines

By the end, you’ll be able to transform raw business data into model-ready inputs efficiently. 🚀

## 1. Handling Missing Data
Missing data can bias or break ML models. Pandas provides multiple methods to inspect and impute missing values.

In [ ]:
import pandas as pd
import numpy as np

# Simulate raw customer data
data = {
    'age': [25, 32, np.nan, 40, 29, np.nan, 38],
    'income': [50000, 60000, 58000, np.nan, 52000, 49000, np.nan],
    'city': ['New York', 'Paris', 'Paris', 'Berlin', 'Berlin', np.nan, 'Tokyo']
}

df = pd.DataFrame(data)
df

### Inspecting Missing Values

In [ ]:
df.isna().sum()

### Imputation Strategies
- **Mean/Median Imputation** for numeric values.
- **Mode/Constant Imputation** for categorical data.
- **Forward/Backward Fill** for time series.

In [ ]:
# Fill numeric columns with median
df['age'] = df['age'].fillna(df['age'].median())
df['income'] = df['income'].fillna(df['income'].mean())

# Fill categorical with mode
df['city'] = df['city'].fillna(df['city'].mode()[0])
df

## 2. Encoding Categorical Variables

Machine learning models work on numbers, not strings. Pandas provides several techniques for encoding categorical features.

In [ ]:
# Label Encoding (simple mapping)
city_map = {city: i for i, city in enumerate(df['city'].unique())}
df['city_encoded'] = df['city'].map(city_map)
df

### One-Hot Encoding using `pd.get_dummies()`

In [ ]:
df_encoded = pd.get_dummies(df, columns=['city'], prefix='city')
df_encoded.head()

## 3. Feature Scaling & Normalization
Scaling ensures that no single feature dominates due to differing scales. This is essential for algorithms like **KNN**, **SVM**, and **Gradient Descent–based models**.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_encoded[['age', 'income']])

df_encoded[['age_scaled', 'income_scaled']] = scaled_features
df_encoded.head()

## 4. Feature Engineering

Feature engineering helps extract hidden patterns from raw data. Let’s demonstrate a few useful transformations.

In [ ]:
# Interaction feature
df_encoded['income_per_age'] = df_encoded['income'] / df_encoded['age']

# Binning continuous variables
df_encoded['age_group'] = pd.cut(df_encoded['age'], bins=[0, 25, 35, 45, 60], labels=['<25', '25-35', '35-45', '45+'])
df_encoded.head()

### Date-Time Features Example

In [ ]:
sales = pd.DataFrame({
    'date': pd.date_range(start='2023-01-01', periods=10, freq='D'),
    'revenue': np.random.randint(100, 1000, 10)
})

sales['day'] = sales['date'].dt.day
sales['weekday'] = sales['date'].dt.day_name()
sales['is_weekend'] = sales['weekday'].isin(['Saturday', 'Sunday'])
sales.head()

## 5. Integrating Pandas with scikit-learn Pipelines

Pandas works seamlessly with scikit-learn for ML preprocessing and modeling.
Let’s build a quick ML-ready preprocessing pipeline.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_features = ['age', 'income']
categorical_features = ['city']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())])

# Mock target variable
y = np.random.randint(2000, 6000, len(df))

# Train pipeline
model.fit(df, y)
pred = model.predict(df)
pred[:5]

## 6. Real-World Problem 1: Predict Employee Salaries

**Scenario:** You have employee demographic data (age, role, experience, city) and want to predict expected salary.

**Steps:**
1. Clean missing experience and city fields.
2. Encode roles and cities with One-Hot encoding.
3. Normalize numeric columns.
4. Train a Linear Regression model.

💡 Use Pandas pipelines and `ColumnTransformer` as above.

## 7. Real-World Problem 2: Customer Churn Prediction

**Scenario:** Telecom company dataset with columns — age, plan, monthly_charge, usage_minutes, churn (target)

**Goal:** Build ML-ready data with Pandas:
- Handle missing usage data.
- Encode plan types.
- Derive new features like charge per minute.
- Output final `X_train`, `y_train` DataFrames ready for model training.

## ⚙️ Under the Hood

- Pandas’ internal block manager efficiently stores mixed dtypes.
- `DataFrame.values` → NumPy arrays, directly used by scikit-learn.
- Transformations like `get_dummies()` and `merge()` preserve alignment by index.
- scikit-learn’s `ColumnTransformer` wraps these transformations for reproducible pipelines.

## ✅ Best Practices / Pitfalls

**Best Practices:**
- Always use `SimpleImputer` and `ColumnTransformer` for consistent preprocessing.
- Prefer **vectorized** feature engineering over loops.
- Keep preprocessing pipelines reproducible using **sklearn Pipelines**.

**Pitfalls:**
- Be cautious when applying `get_dummies()` on unseen test categories.
- Avoid scaling categorical columns.
- Don’t mix fitted transformers across train/test sets.

## 💪 Challenge Exercise

**Task:**
1. Load any public dataset (e.g., Titanic, Housing).
2. Handle missing values using Pandas.
3. Engineer new features (ratios, interactions, flags).
4. Build a scikit-learn pipeline to train a classification model.
5. Compare model performance with and without feature scaling.

_Hint:_ Try experimenting with different encoding schemes (`pd.factorize`, `OneHotEncoder`, `OrdinalEncoder`).